In [4]:
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm import tqdm
from statistics import mean, mode, median
from matplotlib import cm, colors
from distfit import distfit
from inflection import humanize, underscore
import json
from math import sqrt
import statistics


%matplotlib inline

In [7]:
def aparar_arr(arr, porcentagem):
    index_inf = round(len(arr) * porcentagem) - 1
    index_sup = round(len(arr) * (1-porcentagem)) - 1
    return arr[index_inf:index_sup]

In [46]:
def plotHist(dataFrame, metric, path, title, ext):
    data = dataFrame[metric].to_numpy()
    plt.hist(data, bins=np.unique(data), log=True)
    plt.title(title)
    plt.xlabel("Metric value")
    plt.ylabel("Frequency")
    plt.savefig(path+'/hist_'+metric+ ext+'.png')
    plt.close()

In [ ]:
def plotHist(data, metric, path, title, ext):
    plt.hist(data, bins=np.unique(data), log=True)
    plt.title(title)
    plt.xlabel("Metric value")
    plt.ylabel("Frequency")
    plt.savefig(path+'/hist_'+metric+ ext+'.png')
    plt.close()

### Tabela com todos os dados

In [10]:
import pandas as pd
import os
project_path = 'C:/Users/h8men/OneDrive/Documentos/Cefet/TCC/metrics/'
dtframes = []
for file in os.listdir(project_path):
    if file.endswith("class.csv"):
        dfTemp = pd.read_csv(os.path.join(project_path, file))
        dtframes.append(dfTemp)


df = pd.concat(dtframes, axis=0, ignore_index=True)

In [11]:
import functools
from math import sqrt

def calculate_metrics_on_df(dataframe_input, porcentagem):
    dfWithDataOnly = dataframe_input.drop(columns=['file', 'class', 'type', 'assignmen'])
    mean = dfWithDataOnly.mean()
    median = dfWithDataOnly.median()
    mode = dfWithDataOnly.mode().T
    mode['metric'] = mode.index
    std = dfWithDataOnly.std()
    percentile70 = dfWithDataOnly.quantile(0.7)
    percentile90 = dfWithDataOnly.quantile(0.9)

    x = porcentagem
    k = sqrt(-1/(x - 1))
    limite_inf = mean - k*std
    limite_sup = mean + k*std

    meanDf = pd.DataFrame({'metric': mean.index, 'avg': mean.values})
    medianDf = pd.DataFrame({'metric': mean.index,'median': median.values})
    stdDf = pd.DataFrame({'metric': std.index, 'std': std.values})
    modeDf = mode.set_axis(['mode', 'metric'], axis=1)
    percentile70Df = pd.DataFrame({'metric': percentile70.index,'percentile70': percentile70.values})
    percentile90Df = pd.DataFrame({'metric': percentile90.index,'percentile90': percentile90.values})

    limite_infDf = pd.DataFrame({'metric': limite_inf.index, 'limite_inf': limite_inf.values})
    limite_supDf = pd.DataFrame({'metric': limite_sup.index, 'limite_sup': limite_sup.values})
    limites_merged = df_merged = pd.merge(limite_infDf, limite_supDf, on='metric')
    limites_merged['chebyshev'] = list(zip(df_merged['limite_inf'], df_merged['limite_sup']))
    chebyshev = limites_merged.drop(columns=['limite_inf', 'limite_sup'])

    metrics = [meanDf, stdDf, medianDf, modeDf, percentile70Df, percentile90Df, chebyshev]
    df_final = functools.reduce(lambda left, right: pd.merge(left,right,on='metric'), metrics)
    return df_final.sort_values(by='metric')

In [13]:
df_data_ia = calculate_metrics_on_df(df, 0.6)

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
19,abstractMethodsQty,0.055009,0.651543,0.0,0,0.0,0.0,"(-0.9751705801662092, 1.085189175575945)"
43,anonymousClassesQty,0.119011,0.990999,0.0,0.0,0.0,0.0,"(-1.4478963243784315, 1.6859175689972254)"
39,assignmentsQty,14.348358,79.476811,3.0,0.0,9.0,32.0,"(-111.31551447145542, 140.01223098061303)"
0,cbo,6.710141,7.876327,5.0,2,8.0,15.0,"(-5.7434250042778885, 19.16370673747197)"
1,cboModified,9.366113,12.427298,6.0,2,10.0,20.0,"(-10.28316967955523, 29.01539577935909)"
34,comparisonsQty,1.994027,8.792712,0.0,0,1.0,5.0,"(-11.908471909550709, 15.896524995624532)"
27,defaultFieldsQty,0.21842,1.387535,0.0,0,0.0,0.0,"(-1.9754651190925947, 2.412304408900782)"
17,defaultMethodsQty,0.784281,4.755282,0.0,0,0.0,2.0,"(-6.73448030063515, 8.303041321599565)"
5,dit,1.654189,2.480641,1.0,1,2.0,3.0,"(-2.268048254264796, 5.576426377911534)"
2,fanin,2.646496,8.511711,1.0,0,2.0,6.0,"(-10.811700092375684, 16.104692424953313)"


### Tabela com os dados > 1

In [20]:
df_data_ia_without_zeros = calculate_metrics_on_df(df.replace(0, np.NaN), 0.6)
df_data_ia_without_zeros

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
19,abstractMethodsQty,2.427768,3.603013,1.000000,1.0,2.0,5.0,"(-3.269095458830206, 8.12463227726753)"
43,anonymousClassesQty,2.323222,3.748823,1.000000,1.0,2.0,5.0,"(-3.604186723754744, 8.250631278702532)"
39,assignmentsQty,19.288867,91.631096,6.000000,1.0,14.0,40.0,"(-125.59261641843194, 164.17035082018526)"
0,cbo,7.294427,7.948355,5.000000,2.0,8.0,16.0,"(-5.27302490849757, 19.86187918333315)"
1,cboModified,9.953587,12.580825,7.000000,2.0,11.0,20.0,"(-9.93844378376836, 29.84561757911216)"
34,comparisonsQty,6.138124,14.579225,3.000000,1.0,5.0,13.0,"(-16.913653712637295, 29.189902438427307)"
27,defaultFieldsQty,2.479971,4.031514,1.000000,1.0,2.0,5.0,"(-3.894412453722435, 8.854354608458516)"
17,defaultMethodsQty,4.003446,10.126523,2.000000,1.0,3.0,8.0,"(-12.007993622114888, 20.01488524164548)"
5,dit,1.654189,2.480641,1.000000,1.0,2.0,3.0,"(-2.268048254264796, 5.576426377911534)"
2,fanin,4.913965,11.107714,2.000000,1.0,4.0,10.0,"(-12.648872615365883, 22.47680214120247)"


### Tabela com dados aparados 5%

In [ ]:
project_path = '../../Projetos/'
folder = os.listdir(project_path)
metrics = os.listdir(project_path + f'{folder[0]}/')
arr = []
for i in range(len(metrics)):
    metric = metrics[i].replace('.csv', '')
    data = getData(metric,project_path)
    new_data = aparar_arr(data, 0.05)
    values = [metric]
    values.extend(statistic_metric(new_data,0.6))
    arr.append(values)

wmc: 100%|██████████| 44/44 [00:30<00:00,  1.45it/s]


In [ ]:
df_data5 = pd.DataFrame(data=arr, columns=['metric', 'avg', 'std', 'median', 'mode', 'percentile70', 'percentile90', 'chebyshev'])
df_data5

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
0,abstractMethodsQty,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,"(0.0, 0.0)"
1,anonymousClassesQty,0.019674,0.138878,0.000000,0.0,0.000000,0.000000,"(-0.1999115712265472, 0.23926006935786268)"
2,assignmentsQty,4.883057,7.490827,2.000000,0.0,5.000000,15.000000,"(-6.960979844951196, 16.727094365322163)"
3,cbo,7.218092,7.249750,5.000000,0.0,9.000000,18.000000,"(-4.244769332183674, 18.680952601689512)"
4,comparisonsQty,0.604868,1.375443,0.000000,0.0,0.000000,2.000000,"(-1.5698980887061436, 2.779633704449234)"
5,defaultFieldsQty,0.023864,0.152625,0.000000,0.0,0.000000,0.000000,"(-0.21745711826223604, 0.2651847055828654)"
6,defaultMethodsQty,0.227680,0.562670,0.000000,0.0,0.000000,1.000000,"(-0.661979574345274, 1.1173387770398708)"
7,dit,1.504213,0.842605,1.000000,1.0,2.000000,3.000000,"(0.1719371396152478, 2.836488089933038)"
8,fanin,1.444226,2.475302,0.000000,0.0,1.000000,5.000000,"(-2.4695699591554288, 5.358021027304739)"
9,fanout,4.945301,5.107149,3.000000,0.0,7.000000,13.000000,"(-3.1298111717112027, 13.020412380588606)"


### Tabela com dados aparados 10%

In [ ]:
project_path = '../../Projetos/'
folder = os.listdir(project_path)
metrics = os.listdir(project_path + f'{folder[0]}/')
arr = []
for i in range(len(metrics)):
    metric = metrics[i].replace('.csv', '')
    data = getData(metric,project_path)
    new_data = aparar_arr(data, 0.1)
    values = [metric]
    values.extend(statistic_metric(new_data,0.6))
    arr.append(values)

wmc: 100%|██████████| 44/44 [00:22<00:00,  1.99it/s]


In [ ]:
df_data10 = pd.DataFrame(data=arr, columns=['metric', 'avg', 'std', 'median', 'mode', 'percentile70', 'percentile90', 'chebyshev'])
df_data10

,metric,avg,std,median,mode,percentile70,percentile90,chebyshev
0,abstractMethodsQty,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,"(0.0, 0.0)"
1,anonymousClassesQty,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,"(0.0, 0.0)"
2,assignmentsQty,3.707516,4.862352,2.000000,0.0,4.000000,11.000000,"(-3.980536617932502, 11.395569060185291)"
3,cbo,6.469338,5.632936,5.000000,0.0,9.000000,15.000000,"(-2.437116423327888, 15.375792779414054)"
4,comparisonsQty,0.352603,0.785381,0.000000,0.0,0.000000,1.000000,"(-0.8891932701438121, 1.5943996028715566)"
5,defaultFieldsQty,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,"(0.0, 0.0)"
6,defaultMethodsQty,0.126348,0.332241,0.000000,0.0,0.000000,1.000000,"(-0.39897161941958137, 0.6516682194714889)"
7,dit,1.393418,0.660188,1.000000,1.0,1.000000,2.000000,"(0.34956985950404085, 2.437266372003875)"
8,fanin,1.039792,1.565162,0.000000,0.0,1.000000,3.000000,"(-1.4349462151456462, 3.514530954309934)"
9,fanout,4.422419,4.028875,3.000000,0.0,6.000000,11.000000,"(-1.9477920112073024, 10.792629799943352)"


In [21]:
df_data_ia.to_csv('./Statistic_Data_Complete.csv', sep=';')
df_data_ia_without_zeros.to_csv('./Statistic_Data_Without_0.csv', sep=';')

### Histogramas de NOC

In [50]:
hist_path = 'C:/Users/h8men/OneDrive/Documentos/Cefet/TCC/statisticTable/histogram'
plotHist(df, 'noc', hist_path,'Histograma NOC dataset completo', '')

In [53]:
plotHist(df.replace(0, np.NaN), 'noc', hist_path, 'Histograma NOC dataset > 0', '0')

### Tabela com dados
Sem o valor 0, caso a métrica seja discreta e o percentil 70 tenha sido 0 também.

In [ ]:
metrics_without_zero = ['abstractMethodsQty', 'anonymousClassesQty', 'comparisonsQty','defaultFieldsQty', 'defaultMethodsQty','finalMethodsQty', 'innerClassesQty', 'lambdasQty', 'logStatementsQty', 'loopQty', 'mathOperationsQty', 'noc', 'parenthesizedExpsQty', 'privateMethodsQty', 'protectedFieldsQty', 'protectedMethodsQty', 'publicFieldsQty', 'staticFieldsQty', 'staticMethodsQty', 'synchronizedFieldsQty', 'synchronizedMethodsQty', 'tryCatchQty']

In [ ]:
df.to_csv('./Statistic_Data_New_Approach.csv', sep=';', decimal=",")

1 - gerar gráficos para todas as métricas utilizadas pelo Diego
2 - Detalhar metodologia
3 - Fazer as análises e escrever resultados